In [ ]:
!pip install yfinance

In [ ]:
import numpy as np
import yfinance as yf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sc
from ipynb.fs.full.MarkAnalysis import getReturns,Cov_mtrx,PortfolioPerformance,maxSR_data,MinVar_data,MarkPort_data,efficient_frontier
from ipynb.fs.full.MarkAnalysis import max_SR

In [2]:
assets={"Tech":["AAPL MSFT GOOG AMZN META AVGO SAP ACN GLOB"],
        "Banks":["JPM BAC WFC SCHW MS GS C USB PNC"],
        "Mixed":["AAPL MSFT GOOG AMZN META AVGO ACN GLOB WMT"]
        }

start="2022-01-01" #140 tr days. 20 semanas. 7 bloques de 20 dias
end="2022-07-26"


In [ ]:
portfolios = {
        "Tech":["AAPL MSFT GOOG AMZN META AVGO SAP ACN GLOB"],
        "Banks":["JPM BAC WFC SCHW MS GS C USB PNC"],
        "Mixed":["AAPL MSFT GOOG AMZN META AVGO ACN GLOB WMT"],
        "Otros1":["TSLA UNH XOM JNJ V NVDA LLY PG MA"],
        "Otros2":["HD BAC ABBV PFE KO MRK PEP COST ORCL"],
        "Otros3":["TMO MCD CSCO DHR TMUS ABT WFC NEE NKE"],
        "Oil":["SHEL CVX XOM COP TTE EQNR EPD HES"],
        "Food":["MCD MDLZ ADM GIS KHC HSY CMG HRL"]
        }

["2020-01-01","2020-07-23"]
["2020-03-01","2020-09-18"]
["2020-05-01","2020-11-18"]
["2020-07-01","2021-01-21"]
["2020-09-01","2021-03-24"]
["2020-11-01","2021-05-25"]
["2021-01-01","2021-07-26"]
["2021-03-01","2021-09-17"]
["2021-05-01","2021-11-18"]
["2021-07-01","2022-01-20"]
["2021-09-01","2022-03-23"]
["2021-11-01","2022-05-23"]
["2022-01-01","2022-07-26"]

In [4]:
def results(data):
    pred_returns=[]
    real_returns=[]

    for j in range(21,134,7): #Este for permite indexar al primer dia de la prediccion tomando 20 anteriores y hasta el dia 140
        returns= data.pct_change()

        #datos reales
        weekly_returns= returns.iloc[j:j+7,:].mean() #Todas las acciones durante
        #Prediccion
        mean_returns=returns.iloc[j-20:j,:].mean()
        Cov=returns.iloc[j-20:j,:].cov()
        allocation=max_SR(mean_returns, Cov, tr_days=20)[1]
        #Cálculo total de performance
        pred_return_pct=PortfolioPerformance(allocation, mean_returns, Cov, 20)[0]+1
        real_return_pct=np.prod(weekly_returns+1)

        pred_returns.append(pred_return_pct)
        real_returns.append(real_return_pct)

    return pred_returns, real_returns

In [5]:
def week_split(data):
    data1=data.T
    dates=[]
    days=[]
    for j in range(len(data1.columns)): #Creo lista con todas las fechas
        dates.append(str(data1.columns[j])[:10]) #Hay q agarrar hasta :10 porq después tenía la hora
    for j in range(21, len(dates), 7):
        days.append(dates[j])
    return days


In [6]:
def grafico(data, industry):
    y1=np.array(results(data)[0])
    y2=np.array(results(data)[1])
    errores=[]
    for j in range(len(y1)):
        errores.append(abs(round(y1[j]-y2[j],3)*100))
    #print(round(sum(errores)/len(errores),3))
    y1=np.round(y1,2)
    y2=np.round(y2,2)
    xdates=week_split(data)
    x1=np.linspace(1,len(xdates),len(xdates))
    fig, ax=plt.subplots()
    plt.style.use('dark_background')
    plt.plot(x1,y1, color="darkturquoise",label="Prediction")
    plt.plot(x1,y2, color="darkgray",label="Real Value")
    plt.plot(2,0.85, color="white",label="error= "+str(round(sum(errores)/len(errores),3)))
    plt.title(industry)
    ax.set_xlabel("Week")
    ax.set_ylabel("Return")
    plt.legend()
    plt.show()


In [ ]:
FullData=[]
for industry in list(assets.keys()):
    data= getReturns(assets[industry][0], start, end)
    FullData.append(data)

In [ ]:
k=0
for industry in list(assets.keys()):
    grafico(FullData[k], industry)
    k+=1